In [9]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp

In [10]:
@kfp.dsl.pipeline(
  name='WSPipeline',
  description='Workshop ipeline'
  )
def Pipeline():
    vop = dsl.VolumeOp(
        name="volume_creation",
        resource_name="mypvc",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWM
    )
    ddl_step = dsl.ContainerOp(
        name='Ddl',
        image='gcr.io/workshop-267113/workshop:ddl',
        pvolumes={"/mnt/WS":vop.volume}
    )
    train_step = dsl.ContainerOp(
        name='Train',
        image='gcr.io/workshop-267113/workshop:epi3',
        pvolumes={"/mnt/WS":ddl_step.pvolume}
    )

In [11]:
kfp.compiler.Compiler().compile(Pipeline,'WSpipeline.zip')

In [12]:
client = kfp.Client()
my_experiment = client.create_experiment(name='Success')
my_run = client.run_pipeline(my_experiment.id, 'Pipeline','WSpipeline.zip')